<a href="https://colab.research.google.com/github/kdhenderson/msds_colab_notebooks/blob/main/llmWorkflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
import csv


client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.getenv("GROQ_API_KEY"))

question = 'explain quantum computing in simple terms'

completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": question}])


completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": question},
              {"role": "assistant", "content": completion.choices[0].message.content},
              {"role": "user", "content": 'what did I just say?'}])




def load_tweets():
    tweets = []
    with open('data.csv', 'r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            tweets.append(row)
    return tweets

tweets = load_tweets()

# Prompt template for keyword extraction
keyword_prompt = """Given this tweet, assign ONE keyword from the following list that best describes the main theme:
- NATURE
- FOOD
- LEARNING
- FAMILY
- ACHIEVEMENT
- CREATIVITY

Tweet: {tweet}

Return only the keyword, nothing else."""

# Process each tweet
for tweet in tweets[:1]:
    completion = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": keyword_prompt.format(tweet=tweet['tweet'])}]
    )
    keyword = completion.choices[0].message.content.strip()
    tweet['keyword'] = keyword

print(tweet)



print(completion.choices[0].message.content)

# Example of an agentic workflow: Essay Writer and Reviewer

# First agent: Essay Writer
essay_prompt = """Write a one-paragraph essay about the importance of space exploration.
Make it engaging and informative, suitable for a general audience."""

essay_completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": essay_prompt}]
)

essay = essay_completion.choices[0].message.content
print("Essay Writer Agent's Output:")
print("-" * 50)
print(essay)
print("-" * 50)

# Second agent: Essay Reviewer
review_prompt = f"""You are a professional essay reviewer. Review the following one-paragraph essay.
Focus on:
1. Clarity of main argument
2. Use of evidence
3. Writing style
4. Areas for improvement

Essay to review:
{essay}

Provide a structured review with specific examples."""

review_completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": review_prompt}]
)

print("\nEssay Reviewer Agent's Output:")
print("-" * 50)
review = review_completion.choices[0].message.content
print(review)
print("-" * 50)

# Third agent: Essay Reviser
revision_prompt = f"""You are an expert essay writer. You will receive an original essay and a reviewer's feedback.
Your task is to rewrite the essay, carefully addressing all the reviewer's points while maintaining the original topic and scope.

Original essay:
{essay}

Reviewer's feedback:
{review}

Please rewrite the essay, incorporating the reviewer's suggestions. Keep it to one paragraph."""

revision_completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": revision_prompt}]
)
print("\nEssay Reviser Agent's Output (Final Version):")
print("-" * 50)
print(revision_completion.choices[0].message.content)
print("-" * 50)

# Display the full workflow summary
# Fourth agent: 8th Grade Style Adapter
revised_essay = revision_completion.choices[0].message.content

eighth_grade_prompt = f"""You're a cool 8th grader who's really into space! Rewrite this essay in your voice.
Make it sound like you're explaining it to your friends at lunch, but keep the main points.
Use some casual language, maybe a few "like" and "you know," but keep it clear and interesting.
Don't make it too childish - you're 13-14 years old and smart, just more casual and relatable.

Essay to rewrite:
{revised_essay}

Remember:
- Use casual but not silly language
- Keep the main facts and ideas
- Add some enthusiasm (but not too many exclamation points!)
- Maybe mention how this relates to stuff you've seen in movies or games
- Keep it to one paragraph"""

eighth_grade_completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": eighth_grade_prompt}]
)

print("\nFinal 8th Grade Version:")
print("-" * 50)
print(eighth_grade_completion.choices[0].message.content)
print("-" * 50)




# New Workflow: Iterative Research and Edit with Decider
print("\n=== Iterative Research-Edit-Decide Workflow ===\n")

research_topic = "The impact of social media on teenage mental health"
iteration = 1
current_content = ""

def research_agent(topic, current_content):
    research_prompt = f"""You are a thorough researcher. Given the topic and current content (if any),
provide new insights or expand the research. If there's existing content, build upon it rather than starting fresh.

Topic: {topic}
Current Content: {current_content if current_content else 'No content yet - initial research needed'}

Provide 2-3 paragraphs of well-researched information, including:
- Recent statistics or studies
- Expert opinions
- Real-world examples
- Areas that need more exploration"""

    completion = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": research_prompt}]
    )
    return completion.choices[0].message.content

def editor_agent(research_content):
    editor_prompt = f"""You are an expert editor. Review and improve the following research content.
Focus on:
- Clarity and flow
- Logical organization
- Compelling narrative
- Identifying gaps or inconsistencies

Content to edit:
{research_content}

Provide an improved version that maintains accuracy while enhancing readability."""

    completion = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": editor_prompt}]
    )
    return completion.choices[0].message.content

def decider_agent(current_version, iteration):
    decider_prompt = f"""You are the final decision maker. Evaluate this content and decide if it's ready for publication.
Consider:
1. Comprehensiveness of the research
2. Clarity of presentation
3. Logical flow
4. Whether key questions are answered
5. Overall impact

Current Version (Iteration {iteration}):
{current_version}

Respond with:
1. A brief evaluation of the current version
2. Either "DONE" if it's ready for publication, or "CONTINUE" with specific aspects that need more work."""

    completion = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": decider_prompt}]
    )
    return completion.choices[0].message.content

# Run the iterative workflow
while True:
    print(f"\nIteration {iteration}")
    print("-" * 30)

    # Research phase
    print("\nResearcher Agent Working...")
    research_content = research_agent(research_topic, current_content)
    print("Research Output:")
    print(research_content)

    # Editor phase
    print("\nEditor Agent Working...")
    edited_content = editor_agent(research_content)
    print("Edited Version:")
    print(edited_content)

    # Decider phase
    print("\nDecider Agent Evaluating...")
    decision = decider_agent(edited_content, iteration)
    print("Decider's Evaluation:")
    print(decision)

    # Update current content for next iteration
    current_content = edited_content

    # Check if we're done
    if "DONE" in decision.upper():
        print(f"\nProcess completed in {iteration} iterations!")
        print("\nFinal Version:")
        print("-" * 30)
        print(current_content)
        break

    iteration += 1
    if iteration > 5:  # Safety limit
        print("\nReached maximum iterations. Final version:")
        print(current_content)
        break

print("-" * 50)
